In [36]:
import pandas as pd
import sqlalchemy as sa

In [ ]:
# Create the connection string using your credentials
db_uri = "******"

# Create the Engine object
engine = sa.create_engine(db_uri)

#### Test Query

In [38]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, engine)
print(df.head())

      dbn                                        school_name    borough  \
0  27Q260          Frederick Douglass Academy VI High School     Queens   
1  21K559        Life Academy High School for Film and Music   Brooklyn   
2  16K393     Frederick Douglass Academy IV Secondary School   Brooklyn   
3  08X305                               Pablo Neruda Academy      Bronx   
4  03M485  Fiorello H. LaGuardia High School of Music & A...  Manhattan   

  building_code  phone_number    fax_number  grade_span_min  grade_span_max  \
0          Q465  718-471-2154  718-471-2890             9.0              12   
1          K400  718-333-7750  718-333-7775             9.0              12   
2          K026  718-574-2820  718-574-2821             9.0              12   
3          X450  718-824-1682  718-824-1663             9.0              12   
4          M485  212-496-0700  212-724-5748             9.0              12   

  expgrade_span_min expgrade_span_max  ... number_programs  \
0           

### How many schools are there in each borough?

In [41]:
query = """
SELECT  borough, 
        COUNT (DISTINCT dbn) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, engine)
df_result

,borough,school_count
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


### What is the average % of English Language Learners (ELL) per borough?

In [43]:
query = """SELECT 
	-- Select the borough and average ELL percentage
    hsd.borough,
    AVG(sd.ell_percent) avg_percent_eng
FROM 
    nyc_schools.school_demographics sd
LEFT JOIN 
    nyc_schools.high_school_directory hsd
ON 
    sd.dbn = hsd.dbn
GROUP BY 
    hsd.borough -- Groups the data by borough
ORDER BY
    avg_percent_eng DESC; -- Orders the results by the calculated average
    """
pd.read_sql(query, engine)

,borough,avg_percent_eng
0,None,12.708867
1,Manhattan,7.572500


#### Answer: average % of ELL at Manhattan is 7,57. The remaining data lacks a specified borough.

### Using the data from the school demographics and high school directory, write a query to find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)

In [46]:
query = """WITH ranked_schools AS ( -- This is CTE that ranks each school within it's borough
				SELECT 
        hsd.borough,
        hsd.school_name,
        sd.sped_percent,
        -- Use the RANK() window function to create a ranking
        -- PARTITION BY groups the data by borough, so the ranking resets for each borough
        -- ORDER BY sorts schools by the average of sped_percent in descending order
        RANK() OVER (PARTITION BY hsd.borough ORDER BY AVG(sd.sped_percent) DESC) AS rank_in_borough
    FROM 
        nyc_schools.school_demographics sd
    LEFT JOIN 
        nyc_schools.high_school_directory hsd
    ON 
        sd.dbn = hsd.dbn
    GROUP BY 1, 2, 3
)
SELECT 
    -- Select the borough, school name, and the average percentage of special education students
    borough,
    school_name,
    sped_percent
FROM ranked_schools
-- Filter the results from the subquery to only include schools  with a rank of 1, 2, or 3
WHERE 
    rank_in_borough <= 3
-- Final sorting of the results, first by borough and then by the average percentage
ORDER BY 
    borough, 
    sped_percent DESC;
    """
pd.read_sql(query, engine)

,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7
3,None,None,35.5
4,None,None,35.3
5,None,None,32.6


#### Here is the data just for Manhattan, same as in previous query - the remaining data lacks a specified borough.